In [ ]:
from miso.hist_features import get_features
from miso.utils import *
from miso import Miso
from PIL import Image
import pandas as pd
import numpy as np
import scanpy as sc
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
Image.MAX_IMAGE_PIXELS = None
Image.MAX_IMAGE_PIXELS = None
import torch
import random

seed=42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")


In [ ]:
locs = pd.read_csv('./data/10x Visium human tonsil/locs.csv', index_col = 0)


In [ ]:
#Load data and perform necessary preprocessing
rna = sc.read('./data/10x Visium human tonsil/rna.h5ad')
adata=rna.copy()

spatial_coords = locs.iloc[:, [2, 3]].values  
adata.obsm['spatial'] = spatial_coords

rna = preprocess(rna,modality='rna')

protein = sc.read('./data/10x Visium human tonsil/protein.h5ad')
protein = preprocess(protein,modality='protein')

In [ ]:
image_emb=np.load('./data/human tonsil/image_emb.npy')
model = Miso([rna,protein,image_emb],ind_views='all',combs='all',sparse=False,device=device)

model.train()


In [ ]:
clusters = model.cluster(n_clusters=10)
adata.obs['MISO']=clusters

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='miso', title='miso', s=50, show=False)